In [138]:
from tools import logger
from tools.llm import gpt4
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import GitHubIssuesLoader
from tools.extractor import extract_model_from_content
import json
from tqdm import tqdm

In [126]:
def load_docs(repos:list, trim:int=1):
    loaded_repos = []
    docs_list = []
    for repo in repos: 
        loader = GitHubIssuesLoader(
                    repo=repo,
                    include_prs=False
                )
        loaded_repos.append(loader)
        
    for repo in loaded_repos:
        docs = repo.load()
        docs_list.append([docs[i].metadata['title'] for i in range(int(len(docs)/trim))])

    docs_list = [item for sublist in docs_list for item in sublist]
    
    docs_dict = {n: docs_list[n] for n in range(len(docs_list))}
    return docs_dict

In [127]:
issues_dict = load_docs(['langchain-ai/langchain', 'dmlc/xgboost', 'mongodb/mongo', 'sqlalchemy/sqlalchemy', 'karpathy/llm.c'])

In [148]:
test.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,

In [149]:
cats = open('categories.txt', 'r').read()
prompt = PromptTemplate.from_template("Take note of the following categories:\n {categories}. Categorise the following documents based on these categories, with the top three of these categories for each document, as well as a confidence score for each category of a document. Make sure to include the index when you refer to a document : \n {document}")
chain = prompt | gpt4

In [150]:
def get_results(issues, chain, categories:str, prompt:str):
    results = {}
    for key in issues.keys():
        results[key] = chain.invoke(
            {
                "categories": cats, 
                "document": issues[key]
            }
        )
    return results

In [1]:
res = get_results([test[1], test[0], test[2], test[3]], chain, cats, prompt)

In [145]:
out = chain.invoke(
    {
        "categories": cats,
        "documents": [test[0], test[1]]
    }
)

'### Document Analysis and Categorization\n\n#### Document 0: "HF_HUB_OFFLINE environment variable breaks HuggingFaceEndpoints"\n- **Primary Category**: integration_and_customizations\n  - **Key Signals**: The document mentions an environment variable affecting endpoints, suggesting integration or customization issues.\n- **Secondary Category**: platform_dependency\n  - **Key Signals**: The issue with an environment variable could indicate concerns about the flexibility and robustness of the platform, especially in offline or specific configurations.\n- **Tertiary Category**: scalability\n  - **Key Signals**: While not directly mentioned, the underlying issue could affect scalability by limiting how the environment scales in offline or constrained environments.\n- **Urgency Score**: 7\n  - **Rationale**: If the environment variable critically affects functionality, it could hinder development processes or deployments, especially in customized or offline setups.\n- **Confidence Score**:

In [139]:
from typing import List
from pydantic import BaseModel, Field


class IssueLabelCategory(BaseModel):
    """Category of a label with confidence score for a specific github issue."""

    name: str = Field(description="Name of the category. (one of provided)")
    score: float = Field(description="Confidence score for the category.")



class IdentifiedLabels(BaseModel):
    """Dertermined labels with confidence score for a github issue."""

    labels: List[IssueLabelCategory] = Field(description="List of identified labels.")


class ClassifierReadyOutput(BaseModel):
    """Final output format ready for the classifier"""
    index: int = Field(description="The index of the issue")
    labels: IdentifiedLabels = Field(description="Final labels with their respective scores")
    
content = f"""
Take note of the following categories:\n {categories}. 
Categorise the following documents based on these categories, 
with the top three of these categories for each document, and also return an 'urgency' score, as well as a confidence score for each document. 
Make sure to include the index when you refer to a document : \n {documents}"
"""
extracted = extract_model_from_content(model=ClassifierReadyOutput, content=content, llm=gpt4)
# replace content with analysis for reason -> extract. (better quality struct for fine tune)
logger.info(extracted)

OutputParserException: Invalid json output: Your request seems to be missing the specific content details such as the categories and documents to be categorized, including their urgency and confidence scores. Without this information, I'm unable to generate a JSON instance that matches your requirements. 

Please provide the missing details, including the categories, documents, and their respective scores, so I can assist you accordingly.